In [3]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../Data/side_concordance_results.xlsx')

In [4]:
def extract_side(location):
    if pd.isnull(location):
        return 'No Scan'
    
    if 'Left' in location:
        return 'Left'
    elif 'Right' in location:
        return 'Right'
    else:
        return 'Unknown'

In [5]:
def check_spect_sestamibi(row):
    spect = row['SPECT/CT']
    sestamibi = row['Sestamibi']
    
    spect_localizing = pd.notnull(spect) and spect not in ['No Scan', 'Non-localizing']
    sestamibi_localizing = pd.notnull(sestamibi) and sestamibi not in ['No Scan', 'Non-localizing']
    
    if not pd.notnull(spect) and not pd.notnull(sestamibi):
        return 'No SPECT/CT or Sestamibi Recorded'
    elif spect_localizing and sestamibi_localizing:
        return f"SPECT: {spect} | Sestamibi: {sestamibi}"
    elif spect_localizing:
        return f'SPECT/CT: {spect}'
    elif sestamibi_localizing:
        return f'Sestamibi: {sestamibi}'
    else:
        return 'Non-localizing'

In [6]:
preop_imaging_df['SPECT/CT-Sestamibi'] = preop_imaging_df.apply(check_spect_sestamibi, axis=1)
preop_imaging_df['SPECT/CT-Sestamibi'].head()

0    Sestamibi: Right Inferior
1               Non-localizing
2     Sestamibi: Left Inferior
3     Sestamibi: Left Inferior
4              Sestamibi: Left
Name: SPECT/CT-Sestamibi, dtype: object

In [7]:
spectmibi_modality = 'SPECT/CT-Sestamibi'
modality_4dct = '4D CT Scan'

In [8]:
negative_spectmibi_and_4d = preop_imaging_df[
    (preop_imaging_df[spectmibi_modality] == 'Non-localizing') &
    (preop_imaging_df[modality_4dct] == 'Non-localizing')
]

In [9]:
total_cases = negative_spectmibi_and_4d.shape[0]
total_surgical_cure = negative_spectmibi_and_4d[negative_spectmibi_and_4d['Surgical Cure'] == 'Yes'].shape[0]
percentage = (total_surgical_cure / total_cases) * 100 

print(f"{total_cases} cases ({percentage:.2f}%) had both modalities non-localizing but were cured in surgery.")

negative_spect_4d_summary_df = pd.DataFrame({
    'Metric': [
        'Total cases with both modalities non-localizing',
        'Cases cured in surgery',
        'Percent cured within both-negative cases'
    ],
    'Count': [
        total_cases,
        total_surgical_cure,
        f"{percentage:.2f}%"
    ]
})
display(negative_spect_4d_summary_df)

34 cases (97.06%) had both modalities non-localizing but were cured in surgery.


,Metric,Count
0,Total cases with both modalities non-localizing,34
1,Cases cured in surgery,33
2,Percent cured within both-negative cases,97.06%
